In [41]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt
from analysis_functions import make_sure_path_exists

In [42]:
df = pd.read_csv('data/FF2018_data/GlamDataFF2018_Like_Bin_v2.csv') 

In [43]:
df.head()

,Unnamed: 0,subject,trial,rt,choice,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,0,2009,0,0.0,0.0,0.568396,0.431604
1,1,0,1,3371,0,5.0,4.0,0.762332,0.237668
2,2,0,2,1700,1,0.0,5.0,0.446809,0.553191
3,3,0,3,7466,1,1.0,2.0,0.532352,0.467648
4,4,0,4,1889,1,5.0,5.0,0.529736,0.470264


In [44]:
gaze_cols = ([col for col in df.columns
                  if col.startswith('gaze_')])
n_items = len(gaze_cols)

In [45]:
gaze = df[gaze_cols].values
left_gaze_advantage_raw = gaze[:, 0] - np.mean(gaze[:, 1:], axis=1)

df['left_gaze_advantage_raw'] = left_gaze_advantage_raw


In [46]:
left_gaze_advantage_raw

array([ 0.13679245,  0.52466368, -0.10638298, ...,  0.0195003 ,
       -0.23476209, -0.0351706 ])

In [30]:
bins = pd.qcut(left_gaze_advantage_raw, 8,labels=False)
bins

array([5, 7, 2, ..., 4, 1, 3])

In [32]:
np.unique(bins)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [33]:
df['left_gaze_advantage_bins'] = bins


In [64]:
df['left_gaze_advantage_bins'].value_counts()

7    495
5    495
3    495
1    495
6    495
4    495
2    495
0    495
Name: left_gaze_advantage_bins, dtype: int64

## First attempt to bin gaze difference (left_gaze_advantage) by participant using z-score

In [60]:
bins_values = []
for i in (df['subject'].unique()):
    Choicedata_gaze = df.loc[df['subject'] == i]
    bins_per_subj = pd.qcut(Choicedata_gaze['left_gaze_advantage_raw'], 8,labels=False)
    for  j in range(len(bins_per_subj)):    
        bins_values.append(bins_per_subj.values[j])
df['left_gaze_advantage_bins'] = bins_values


In [63]:
df

,Unnamed: 0,subject,trial,rt,choice,item_value_0,item_value_1,gaze_0,gaze_1,left_gaze_advantage_raw,left_gaze_advantage_bins
0,0,0,0,2009,0,0.0,0.0,0.568396,0.431604,0.136792,5
1,1,0,1,3371,0,5.0,4.0,0.762332,0.237668,0.524664,7
2,2,0,2,1700,1,0.0,5.0,0.446809,0.553191,-0.106383,1
3,3,0,3,7466,1,1.0,2.0,0.532352,0.467648,0.064704,4
4,4,0,4,1889,1,5.0,5.0,0.529736,0.470264,0.059473,3
5,5,0,5,2227,1,4.0,4.0,0.370529,0.629471,-0.258943,0
6,6,0,6,4259,0,4.0,0.0,0.482254,0.517746,-0.035493,2
7,7,0,7,3505,0,3.0,0.0,0.670739,0.329261,0.341478,7
8,8,0,8,2548,0,3.0,0.0,0.528497,0.471503,0.056995,3
9,9,0,9,3226,1,0.0,5.0,0.581579,0.418421,0.163158,5


In [ ]:
z_matrix_values = []
for i in (df['subject'].unique()):
    Choicedata_ddm = df.loc[df['parcode'] == i]
    
    pX_LV =  pd.to_numeric(Choicedata_ddm["item_left"]) 
    meanLV = np.mean(pX_LV)
    stdLV =  np.std(pX_LV)
    p_zLV = (pX_LV - meanLV)/stdLV
     
    # We generate bins using the z-scored values.
    # We consider 6 levels
    # 1 . -inf < x <  -2*std
    # 2 . -2*std <= x <  -std
    # 3 . -std <= x <  0
    # 4 . 0 <= x <  std
    # 5 . std <= x <  2*std
    # 6 . 2*std <= x <  inf
    
    binLV =  -1
    p_binLV = []
    
    
    # Avoid the problem of indexing
    p_zLV = p_zLV.reset_index()    
    p_zLV = p_zLV.drop(['index'], 1)
    p_zLV = p_zLV["item_left"]

    # 6 bins for the value
    for  j in range(len(p_zLV)):  
        # bin the value of the left item 
        if p_zLV[j] < (-2*stdLV):
            binLV = 0
        elif -2*stdLV <= p_zLV[j] and p_zLV[j]< -stdLV:
            binLV = 1
        elif -stdLV <= p_zLV[j] and p_zLV[j]< 0:
            binLV = 2
        elif 0 <= p_zLV[j] and p_zLV[j]< stdLV:
            binLV = 3
        elif stdLV <= p_zLV[j] and p_zLV[j]< 2*stdLV:
            binLV = 4
        elif 2*stdLV <=p_zLV[j]:
            binLV = 5
        
        # bin the value of the left item 
        if p_zRV[j] < -2*stdRV:
            binRV = 0
        elif -2*stdRV <= p_zRV[j] and p_zRV[j]< -stdRV:
            binRV = 1
        elif -stdRV <= p_zRV[j] and p_zRV[j]< 0:
            binRV = 2
        elif 0 <= p_zRV[j] and p_zRV[j]< stdRV:
            binRV = 3
        elif stdRV <= p_zRV[j] and p_zRV[j]< 2*stdRV:
            binRV = 4
        elif 2*stdRV < p_zRV[j]:
            binRV = 5
        
        p_binLV.append(binLV)
        p_binRV.append(binRV)
    
    z_matrix_aux= np.column_stack((p_zLV,p_zRV,p_binLV,p_binRV))
    
    for  j in range(len(z_matrix_aux)):    
        z_matrix_values.append(z_matrix_aux[j])

In [ ]:
def add_left_gaze_advantage(df):
    """
    Compute gaze advantage of left item and add to DataFrame.

    Left relative gaze – mean other relative gaze
    In the binary case, this reduces to g0 - g1.

    Parameters
    ----------
    df :      <pandas DataFrame>
              Trial wise DataFrame containing columns for gaze_i
    """

    # infer number of items
    gaze_cols = ([col for col in df.columns
                  if col.startswith('gaze_')])
    n_items = len(gaze_cols)

    gaze = df[gaze_cols].values
    left_gaze_advantage = gaze[:, 0] - np.mean(gaze[:, 1:], axis=1)

    df['left_gaze_advantage'] = left_gaze_advantage

    return df.copy()